### Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from IPython.display import Image, display

import constants, utils
from multi_agent_graph import get_multi_agent_summarizer_graph
from data_classes.book_summarizer_state import BookSummarizerState

### Execute the Graph


In [4]:
config = {"recursion_limit": 50}
inputs = {
    "book_name": constants.BOOK_NAME, 
    "book_dir": constants.BOOK_PATH, 
    "num_chapters_to_summarize": constants.NUM_CHAPTERS_TO_SUMMARIZE, 
    "verbose": constants.VERBOSE,
    "summaries_dir": constants.BOOK_SUMMARIES_DIR,
}

graph = get_multi_agent_summarizer_graph()
app = graph.compile()
# display(Image(app.get_graph(xray=True).draw_mermaid_png())) 

book_config = BookSummarizerState(**inputs)
for event in app.stream(book_config.dict(), config=config):
    print("Event:", event)




 33 chapters found in the book
Event: {'Epub Loader': {'book_name': 'eat_that_frog', 'verbose': 0, 'book_dir': '/home/pedro/Documents/GitHub/SwarmGPT/data/books/eat_that_frog.epub', 'book': <book_summarizer.data_classes.epub.EPUB object at 0x7f1b09740730>, 'chapter_titles_all': ['Cover Page', 'Title Page', 'Copyright Page', 'Dedication', 'Contents', 'Preface', 'Introduction: Eat That Frog', '1 Set the Table', '2 Plan Every Day in Advance', '3 Apply the 80/20 Rule to Everything', '4 Consider the Consequences', '5 Practice Creative Procrastination', '6 Use the ABCDE Method Continually', '7 Focus on Key Result Areas', '8 Apply the Law of Three', '9 Prepare Thoroughly Before You Begin', '10 Take It One Oil Barrel at a Time', '11 Upgrade Your Key Skills', '12 Identify Your Key Constraints', '13 Put the Pressure on Yourself', '14 Motivate Yourself into Action', '15 Technology Is a Terrible Master', '16 Technology Is a Wonderful Servant', '17 Focus Your Attention', '18 Slice and Dice the Ta